In [1]:
import importlib
import local_utils; importlib.reload(local_utils)
from local_utils import *

train_df = pd.read_csv(DATA_DIR + '/train_v2.csv')
test_df = pd.read_csv(DATA_DIR + '/sample_submission_v2.csv')

label_map, inv_label_map, Y = process_labels(train_df)
print("Shape of Y {}".format(Y.shape))

train_inx, valid_inx = stratified_sampling(Y)
print("Train {}; Valid {};".format(len(train_inx), len(valid_inx)))

w_size = 256
h_size = 256
input_shape = (w_size, h_size, 3)
print(input_shape)

Using TensorFlow backend.


Import time: 10.713954210281372
Import time: 5.5789947509765625e-05
Shape of Y (40479, 17)
Train 36431; Valid 4048;
(256, 256, 3)


In [ ]:
start = time()
X = load_images(train_df['image_name'].values, DATA_DIR + '/train-jpg', True, w_size, h_size)
print(X.shape)
print(time() - start)

In [2]:
from keras.applications.resnet50 import ResNet50
resnet50_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)

resnet50_model.layers.pop()
resnet50_model.outputs = [resnet50_model.layers[-1].output]
resnet50_model.layers[-1].outbound_nodes = []
resnet50_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 256, 256, 3)   0                                            
____________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D) (None, 262, 262, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Conv2D)                   (None, 128, 128, 64)  9472        zero_padding2d_1[0][0]           
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 128, 128, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [4]:
resnet50_train_feat = resnet50_model.predict(X, batch_size=16, verbose = 1)
save_array(DATA_DIR + '/models/resnet50_train_feat.dat', resnet50_train_feat)

40479/40479 [==============================] - 564s   


In [3]:
# del X
# del resnet50_train_feat

start = time()
X_test = load_images(test_df['image_name'].values, DATA_DIR + '/test-jpg', True, w_size, h_size)
print(X_test.shape)
print(time() - start)

(30000, 256, 256, 3)
27.798438787460327


In [4]:
resnet50_test_feat = resnet50_model.predict(X_test, batch_size=16, verbose = 1)

30000/30000 [==============================] - 423s   


In [6]:
save_array(DATA_DIR + '/models/resnet50_test_feat.dat', resnet50_test_feat)

In [ ]:
# resnet50_train_feat = load_array(DATA_DIR + '/models/resnet50_train_feat.dat')
# print(resnet50_train_feat.shape)

# resnet50_test_feat = load_array(DATA_DIR + '/models/resnet50_test_feat.dat')
# print(resnet50_test_feat.shape)